In [1]:
from snowflake.snowpark import Session
from snowflake.core import Root
from snowflake.core.database import Database
from snowflake.core.schema import Schema

In [2]:
session = Session.builder.config("connection_name","myconnection").create()
root = Root(session)

c:\Users\powel\anaconda3\envs\.venv-sp500-stocks-app\lib\site-packages\snowflake\connector\config_manager.py:344: UserWarning: Bad owner or permissions on ..\.snowflake\connections.toml
  warn(f"Bad owner or permissions on {str(filep)}{chmod_message}")


In [3]:
## Load the info table FROM the database into a Snowflake DataFrame
sp500_info_df = session.table("investment_db.investment_schema.sp500_info")
sp500_info_df.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## SELECTING DATA

In [4]:
## How to filter the data (i.e. WHERE clause)
from snowflake.snowpark.functions import col
sp500_info_df_AMZN = sp500_info_df.filter(col('"symbol"') == "AMZN")
sp500_info_df_AMZN.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [6]:
## How to select specific columns (i.e. SELECT clause) - FOUR METHODS
sp500_info_df_SELECT = sp500_info_df.select(col('symbol'), col('longname'), col('sector'), col('industry'), col('col_52weekchange'), col('sandp52weekchange'))

# sp500_info_df_SELECT = sp500_info_df.select('"symbol"', '"longName"', '"sector"', '"industry"', '"52WeekChange"', '"SandP52WeekChange"')

# sp500_info_df_SELECT = sp500_info_df.select(sp500_info_df['"symbol"'], sp500_info_df['"longName"'], sp500_info_df['"sector"'], sp500_info_df['"industry"'], sp500_info_df['"52WeekChange"'], sp500_info_df['"SandP52WeekChange"'])

# sp500_info_df_SELECT = sp500_info_df.select(sp500_info_df.symbol, sp500_info_df.longName, sp500_info_df.sector, sp500_info_df.industry, sp500_info_df.52WeekChange, sp500_info_df.SandP52WeekChange)

sp500_info_df_SELECT.show()

SnowparkSQLException: (1304): 01b10820-0004-df72-0002-99570001d132: 000904 (42000): SQL compilation error: error line 1 at position 7
invalid identifier 'SYMBOL'

## JOINING DATA

In [ ]:
from snowflake.snowpark.functions import col

In [ ]:
## Load the returns table FROM the database into a Snowflake DataFrame
sp500_returns_df = session.table("investment_db.investment_schema.sp500_returns")
sp500_returns_df.show()

In [ ]:
## View the info table we loaded earlier
sp500_info_df.show()

In [ ]:
## Join the info and returns tables
sp500_joined_dfs = sp500_returns_df.join(sp500_info_df, col('TICKER') == col('"symbol"'))
sp500_joined_dfs.show()

In [ ]:
## We want to join the two tables together on the symbol column